# UBS_lauzhack - Entity resolution model

## 1. Pre-processing

Observed the given dataset and apply the proper transformations to the data in order to better anlyse it and create the model.

In [102]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
import warnings


InteractiveShell.ast_node_interactivity = "all"
#pd.set_option('precision', 3)
display.precision = 3
sns.set()
warnings.filterwarnings('ignore')

# extra imports
from pandas import read_csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
#from statsmodels.genmod.generalized_linear_model import GLM
from pandas.plotting import scatter_matrix
from scipy.stats import boxcox



In [103]:
# Read the CSV files from the 'data' subfolder
account_booking_df = pd.read_csv('data/account_booking_train.csv')
external_parties_df = pd.read_csv('data/external_parties_train.csv')

In [104]:
# Combine the tables with merge using 
combined_data = pd.merge(account_booking_df, external_parties_df, on='transaction_reference_id', how='inner')

In [105]:
#Null count of each variables to know which ones are more relevant to analyze
combined_data.isnull().sum()

transaction_reference_id            0
debit_credit_indicator              0
account_id                          0
transaction_amount                  0
transaction_currency                0
transaction_date                    0
party_role                          0
party_info_unstructured             0
parsed_name                         0
parsed_address_street_name        739
parsed_address_street_number     3030
parsed_address_unit             11064
parsed_address_postal_code       3480
parsed_address_city              1168
parsed_address_state             9107
parsed_address_country           6490
party_iban                       3399
party_phone                      5000
external_id                         0
dtype: int64

In [106]:
# Drop the columns that have more than 50% of missing values and irrelevant information

## External parties data

irrelevant_cols_external = ['party_info_unstructured', 'parsed_address_unit', 'parsed_address_state', 'parsed_address_country']
external_parties_df.drop(columns=irrelevant_cols_external, inplace=True, errors='ignore')

## Accounts booking data
duplicate_ids = account_booking_df[account_booking_df.duplicated(subset='transaction_reference_id', keep=False)]
account_booking_df = account_booking_df[~account_booking_df['transaction_reference_id'].isin(duplicate_ids['transaction_reference_id'])]

irrelevant_cols_booking = ['debit_credit_indicator']
account_booking_df.drop(columns=irrelevant_cols_booking, inplace=True, errors='ignore')

# Merge the two dataframes

merged_df = pd.merge(external_parties_df, account_booking_df, on='transaction_reference_id', how='inner')

merged_df.to_csv('merged_data_cleaned.csv', index=False)

### 1.1 Pre-processing of the variable parsed_name

In [107]:
# Delete honorifics from the names


def delete_honorifics(name): 
    name['parsed_name'] = name['parsed_name'].str.replace('Mr. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Ms. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Mrs. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Miss ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Dr. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Prof. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Rev. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('Hon. ', '')

    name['parsed_name'] = name['parsed_name'].str.replace('mr. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('ms. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('mrs. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('miss ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('dr. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('prof. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('rev. ', '')
    name['parsed_name'] = name['parsed_name'].str.replace('hon. ', '')

delete_honorifics(merged_df)
print(merged_df['parsed_name'])

0                 mary mith
1               yesneia kim
2           w. roberson jr.
3         azquez-nelson co.
4             m.j. bytd iii
                ...        
11059    james alvarado jr.
11060            marcnguyen
11061          joseph davis
11062           gonzalezltd
11063        simmons-conway
Name: parsed_name, Length: 11064, dtype: object


In [109]:
# Delete duplicates in the names

# Function to delete duplicated words in names
def delete_duplicates(name):
    def remove_duplicates(text):
        words = text.split()
        seen = set()
        result = []
        for word in words:
            if word.lower() not in seen:
                seen.add(word.lower())
                result.append(word)
        return ' '.join(result)
    
    name['parsed_name'] = name['parsed_name'].apply(remove_duplicates)

delete_duplicates(merged_df) 
print(merged_df['parsed_name'])

0                 mary mith
1               yesneia kim
2           w. roberson jr.
3         azquez-nelson co.
4             m.j. bytd iii
                ...        
11059    james alvarado jr.
11060            marcnguyen
11061          joseph davis
11062           gonzalezltd
11063        simmons-conway
Name: parsed_name, Length: 11064, dtype: object


In [124]:
import pandas as pd
import numpy as np
from metaphone import doublemetaphone

# Función para aplicar Metaphone
def apply_metaphone(value):
    if isinstance(value, list) or isinstance(value, np.ndarray):  
        return np.nan  
    if pd.isna(value) or not isinstance(value, str):  
        return np.nan
    return doublemetaphone(value.strip().lower())[0]  


def split_and_metaphone(full_name):
    """
    Divide un nombre completo en sus partes (palabras), aplica Metaphone a cada parte,
    y las junta nuevamente con espacios.
    """
    if pd.isna(full_name) or not isinstance(full_name, str):  
        return np.nan

    parts = full_name.strip().split()

    return ' '.join(apply_metaphone(part) for part in parts)

# Delete irrelevant columns

irrelevant_cols_external = ['party_info_unstructured', 'parsed_address_unit', 'parsed_address_state', 'parsed_address_country']
merged_df.drop(columns=irrelevant_cols_external, inplace=True, errors='ignore')

duplicate_ids = account_booking_df[account_booking_df.duplicated(subset='transaction_reference_id', keep=False)]
merge_df = merged_df[~merged_df['transaction_reference_id'].isin(duplicate_ids['transaction_reference_id'])]

irrelevant_cols_booking = ['debit_credit_indicator', 'transaction_amount', 'transaction_currency']
merge_df.drop(columns=irrelevant_cols_booking, inplace=True, errors='ignore')


# Apply Metaphone to the names and addresses

merged_df['parsed_name'] = merged_df['parsed_name'].apply(split_and_metaphone)

merged_df['parsed_address_street_name'] = merged_df['parsed_address_street_name'].apply(apply_metaphone)

merged_df.to_csv('merged_data_cleaned2.csv', index=False)

print(merged_df['parsed_name'].head(20))


0                MR M
1              ASN KM
2            RPRSN JR
3          ASKSNLSN K
4             MJ PT A
5             FLRS LT
6         KRSTFR KTRS
7             KL KLRK
8        FLPS ANKRPRT
9     KR FKSR ANT TRM
10           RPRSN JR
11           MKL AKLR
12             ATM KL
13           M STFNSN
14              SN RL
15             NT PLK
16               KRMN
17            TFT ARK
18           RMNT JNS
19             TMSPKR
Name: parsed_name, dtype: object


### 1.2 Phone pre-processing

In [ ]:
# Phone number normalization

combined_data['party_phone'] = combined_data['party_phone'].str.split('x').str[0]
combined_data['party_phone'] = combined_data['party_phone'].replace(r'\D', '', regex=True)
combined_data['party_phone'] = combined_data['party_phone'].str.lstrip('0')

data = combined_data[['party_phone']]
print(data)

          party_phone
0                 NaN
1        419477655328
2                 NaN
3       4119006262567
4                 NaN
...               ...
11059  13445497715686
11060   4116282024224
11061             NaN
11062             NaN
11063  51473539600789

[11064 rows x 1 columns]


## 2. Model?

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Sample data for demonstration
data = pd.DataFrame({
    'transaction_reference_id': [1, 2, 3, 4, 5],
    'parsed_name': ['John Doe', 'Jon Doe', 'Jane Doe', 'John Smith', 'Jane Smith'],
    'postal_code': ['12345', '12345', '67890', '12345', '67890'],
    'feature1': [10, 20, 30, 40, 50],
    'feature2': [1, 2, 3, 4, 5],
    'label': [0, 1, 0, 1, 0]
})

# Prepare the data
X = data[['feature1', 'feature2']]
y = data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define parameters
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data], early_stopping_rounds=10)

# Predict
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_binary)
conf_matrix = confusion_matrix(y_test, y_pred_binary)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)

In [ ]:


# Paso 3: Construcción del grafo
graph = nx.Graph()

# Agregar nodos
for idx, row in merged_data_cleaned2.iterrows():
    node_id = row['transaction_reference_id']
    graph.add_node(node_id, attributes=row)

# Agregar aristas basadas en similitud
for node1, node2 in possible_pairs:  # Comparar dentro de cada bloque
    sim_name = jaro_winkler_similarity(graph.nodes[node1]['attributes']['parsed_name'],
                                       graph.nodes[node2]['attributes']['parsed_name'])
    if sim_name > 0.8:
        graph.add_edge(node1, node2)

# Paso 4: Identificar componentes conexos
clusters = list(nx.connected_components(graph))

# Paso 5: Asignar IDs de cluster
cluster_mapping = {node: cluster_id for cluster_id, cluster in enumerate(clusters) for node in cluster}
data['external_id'] = data['transaction_reference_id'].map(cluster_mapping)


NameError: name 'possible_pairs' is not defined